In [3]:
import os
#os.system('pwd')
import re
import pandas as pd
from collections import defaultdict
from sympy import symbols, simplify_logic, Or
from sympy.logic.boolalg import And, Or, Not, simplify_logic, SOPform
from itertools import product



In [ ]:
#merge the generated mif files from the platform
#directory to the generated memory
#
def merge_mif_files(directory):
    # Dictionary to store file groups based on second index
    file_groups = defaultdict(list)

    # Regular expression to extract {index}_{subindex}.mif pattern
    pattern = re.compile(r"(\d+)_(\d+)\.mif")

    # Get all .mif files and categorize them based on their second index
    for filename in os.listdir(directory):
        if filename.endswith(".mif"):
            match = pattern.match(filename)
            if match:
                first_index, second_index = match.groups()
                file_groups[second_index].append(filename)

{'1': ['0_1.mif', '2_1.mif', '1_1.mif', '5_1.mif', '9_1.mif', '4_1.mif', '3_1.mif', '8_1.mif', '7_1.mif', '6_1.mif'], '0': ['8_0.mif', '2_0.mif', '1_0.mif', '3_0.mif', '7_0.mif', '5_0.mif', '9_0.mif', '6_0.mif', '0_0.mif', '4_0.mif']}


['CHVs_1.mif', 'CHVs_0.mif']


In [ ]:
#merge the generated mif files from the platform
def merge_mif_files(directory):
    # Dictionary to store file groups based on second index
    file_groups = defaultdict(list)

    # Regular expression to extract {index}_{subindex}.mif pattern
    pattern = re.compile(r"(\d+)_(\d+)\.mif")

    # Get all .mif files and categorize them based on their second index
    for filename in os.listdir(directory):
        if filename.endswith(".mif"):
            match = pattern.match(filename)
            if match:
                first_index, second_index = match.groups()
                file_groups[second_index].append(filename)

    # Sort file groups by second index
    for second_index, files in file_groups.items():
        files.sort(key=lambda x: int(x.split("_")[0]))  # Sort by first index

        # Define output file
        output_file = os.path.join(directory, f"CHV_{second_index}.mif")

        # Merge files
        with open(output_file, "w", encoding="utf-8") as outfile:
            for file in files:
                file_path = os.path.join(directory, file)
                with open(file_path, "r", encoding="utf-8") as infile:
                    content = infile.read().strip()  # Read and strip spaces/newlines
                    outfile.write(content + "\n")  # Write content as a single line



# make memory files
def groupMifFiles(directory):
    files = [f for f in os.listdir(directory) if f.endswith(".mif")]

    grouped_files = {}
    for file in files:
        class_index, index = file.replace(".mif", "").split("_")
        grouped_files.setdefault(index, []).append(file)

    # print (grouped_files)

    for index, file_list in grouped_files.items():
        output_file = os.path.join(directory, f"CHVs_{index}.mif")
        with open(output_file, "w") as outfile:
            for filename in sorted(file_list, key=lambda x: int(x.split("_")[0])):
                with open(os.path.join(directory, filename), "r") as infile:
                    outfile.write(infile.read().strip() + "\n")
                    
def read_mif_file(file_path):
    bitvectors = []
    with open(file_path, 'r') as file:
        for line in file:
            lines = line.strip()
            if lines:  # Skip empty lines
                bitvectors.append(lines)
    return bitvectors

def write_mif_file(file_path, j , bitvector):
    with open(file_path+"_"+str(j)+".mif", 'w') as file:
        file.write(f"{''.join(bitvector)}")


def generate_flag_memory_file (bitvectors):
    # Transpose bitvectors to analyze each bit position
    num_bitstreams = len(bitvectors)        # Number of bitstreams (outermost list)
    num_rows = len(bitvectors[0])          # Number of rows in each bitstream
    num_cols = len(bitvectors[0][0])       # Number of columns in each row
    
    result = []
    # transposed_bits = zip(*bitvectors)
    # print(transposed_bits)
    for row in range(num_rows):
        flag_row = []
        for col in range(num_cols):
            # Collect elements from all bitstreams at position [row][col]
            bits = [bitvectors[stream][row][col] for stream in range(num_bitstreams)]
            # Check if all elements are the same
            flag_row.append('1' if len(set(bits)) == 1 else '0')
        # Append the flag row to the result
        result.append(''.join(flag_row)) 
        
    # flag_vector = ['0' if len(set(bits)) == 1 else '1' for bits in transposed_bits]
    return result

def generate_parity_memory_file(bitvectors, flag_vector):
    parity_vector = []
    result = []
    for i in range(len(flag_vector)):
        if flag_vector[i] == '1':
                column = [bv[i] for bv in bitvectors]
                parity = column.count('1') % 2
                parity_vector.append(str(parity))
        # print("parity is : ", parity_vector)
    result.append(''.join(parity_vector))
    # print(result)
    return result

def Sizing_the_same(file_name, directory):
    file_data = []
    for filename in os.listdir():
        if filename.startswith(file_name):
            with open(filename, 'r') as file:
                content = file.read().strip()
                file_data.append((filename, content))
    # print(file_data)
    # Find the maximum length of the file contents
    max_length = max(len(content) for _, content in file_data)

    # Pad the contents of shorter files with leading zeros and save the changes
    for filename, content in file_data:
        if len(content) < max_length:
            padding_needed = max_length - len(content)
            padded_content = '0' * padding_needed + content
            with open(os.path.join(directory, filename), 'w') as file:
                file.write(padded_content)
        else:
            with open(os.path.join(directory, filename), 'w') as file:
                file.write(content)

# to make efficent hardware for ECC
# Function to read a MIF file and process its content
def process_mif_file(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()

    # Grouping values by occurrences in the file
    value_groups = defaultdict(list)
    
    for index, line in enumerate(lines):
        value = line.strip()
        value_groups[value].append(index)

    # Convert to dictionary
    grouped_data = dict(value_groups)

    # Determine the highest index for binary conversion
    max_index = max(max(indexes) for indexes in grouped_data.values())
    binary_length = len(bin(max_index)[2:])  # Get binary length

    # Convert line indexes to binary
    for key in grouped_data:
        grouped_data[key] = [format(index, f'0{binary_length}b') for index in grouped_data[key]]

    return grouped_data, binary_length

# Function to generate VHDL ROM-based LUT code
def generate_vhdl_rom(file_name, grouped_data, binary_length):
    vhdl_code = f"""
library IEEE;
use IEEE.STD_LOGIC_1164.ALL;

entity ROM_LUT_{file_name} is
    Port (
        address : in STD_LOGIC_VECTOR({binary_length-1} downto 0);
        data_out : out STD_LOGIC_VECTOR(4 downto 0) -- 5-bit value
    );
end ROM_LUT_{file_name};

architecture Behavioral of ROM_LUT_{file_name} is
begin
    process(address)
    begin
        case address is
    """
    
    for value, indexes in grouped_data.items():
        for index in indexes:
            vhdl_code += f'            when "{index}" => data_out <= "{value}";\n'

    vhdl_code += """
            when others => data_out <= "00000"; -- Default case
        end case;
    end process;
end Behavioral;
    """
    
    return vhdl_code

def generate_vhdl_sop(file_name, grouped_data, binary_length):
    num_output_bits = len(next(iter(grouped_data.keys())))  # Determine output bit width
    variables = [symbols(f'A{i}') for i in range(binary_length)]  # Define input variables
    simplified_expressions = {}

    # Generate simplified SOP for each output bit
    for bit_pos in range(num_output_bits):
        boolean_expressions_bit = []
        
        for value, binary_indexes in grouped_data.items():
            bit_value = int(value[bit_pos])
            if bit_value == 1:
                for binary_index in binary_indexes:
                    terms = [variables[i] if bit == '1' else Not(variables[i]) for i, bit in enumerate(binary_index)]
                    boolean_expressions_bit.append(And(*terms))  # Use AND for product terms

        # Simplify SOP expression
        if boolean_expressions_bit:
            combined_expression = Or(*boolean_expressions_bit)  # Use OR for sum of products
            simplified_expressions[f"V{bit_pos}"] = simplify_logic(combined_expression, form='dnf')

    # Generate VHDL code for SOP logic
    vhdl_code = f"""
library IEEE;
use IEEE.STD_LOGIC_1164.ALL;

entity SOP_LUT_{file_name} is
    Port (
        A : in STD_LOGIC_VECTOR({binary_length-1} downto 0);
        V : out STD_LOGIC_VECTOR({num_output_bits-1} downto 0)
    );
end SOP_LUT_{file_name};

architecture Behavioral of SOP_LUT_{file_name} is
begin
"""

    # Convert simplified SOP expressions to VHDL syntax
    for bit, expression in simplified_expressions.items():
        vhdl_expr = str(expression).replace('~', 'not ').replace('&', ' and ').replace('|', ' or ')
        vhdl_code += f"    V({bit[-1]}) <= {vhdl_expr};\n"

    vhdl_code += "end Behavioral;\n"

    return vhdl_code


# Function to generate minimized SOP-based VHDL with don't cares
def generate_vhdl_sop_with_dont_care(file_name, grouped_data, binary_length):
    num_output_bits = len(next(iter(grouped_data.keys())))  # Determine output bit width
    variables = [symbols(f'A{i}') for i in range(binary_length)]  # Define input variables
    simplified_expressions = {}

    # Generate all possible binary indexes (0 to 2^binary_length - 1)
    all_indexes = {format(i, f'0{binary_length}b') for i in range(2**binary_length)}
    
    # Determine used indexes and don't care indexes
    used_indexes = {index for indexes in grouped_data.values() for index in indexes}
    dont_care_indexes = all_indexes - used_indexes  # All missing indexes are don't cares

    # Generate minimized SOP for each output bit
    for bit_pos in range(num_output_bits):
        minterms = []
        dont_cares = []
        
        for value, binary_indexes in grouped_data.items():
            bit_value = int(value[bit_pos])
            if bit_value == 1:  # Add minterms for '1' values
                for binary_index in binary_indexes:
                    minterms.append(tuple(int(bit) for bit in binary_index))

        # Convert don't care conditions to tuple format for SOPform minimization
        for binary_index in dont_care_indexes:
            dont_cares.append(tuple(int(bit) for bit in binary_index))

        # Simplify SOP expression using ESPRESSO (handles don't cares correctly)
        if minterms:
            simplified_expr = SOPform(variables, minterms, dont_cares)
            simplified_expressions[f"V{bit_pos}"] = simplified_expr

    # Generate VHDL code for SOP logic
    vhdl_code = f"""
library IEEE;
use IEEE.STD_LOGIC_1164.ALL;

entity SOP_LUT_{file_name} is
    Port (
        A : in STD_LOGIC_VECTOR({binary_length-1} downto 0);
        V : out STD_LOGIC_VECTOR({num_output_bits-1} downto 0)
    );
end SOP_LUT_{file_name};

architecture Behavioral of SOP_LUT_{file_name} is
begin
"""

    # Convert simplified SOP expressions to VHDL syntax
    for bit, expression in simplified_expressions.items():
        vhdl_expr = str(expression).replace('~', 'not ').replace('&', ' and ').replace('|', ' or ')
        vhdl_code += f"    V({bit[-1]}) <= {vhdl_expr};\n"

    vhdl_code += "end Behavioral;\n"

    return vhdl_code

In [11]:
def main():
    # File paths
    directory = './hardware/RCEHDC/mnist_example/mem/normal/'
    output_dir = './hardware/RCEHDC/mnist_example/mem/normal/'
    groupMemFiles = [f for f in os.listdir(directory) if f.startswith("CHV") and f.endswith(".mif")]
    groupECCMem = [f for f in os.listdir(directory) if f.startswith("ECC_memory_file_") and f.endswith(".mif")]
    # Read the bitvectors from CHV_img.mif
    print(groupECCMem)
    groupMemFiles.reverse()
    for i, eccMem in enumerate(groupECCMem):
        grouped_data, binary_length = process_mif_file(directory+eccMem)
        # print(grouped_data, binary_length) 
        file_name = str(i)
        print(i , eccMem)
        
        vhdl_code = generate_vhdl_sop(file_name, grouped_data, binary_length)
        vhdl_file_path = os.path.join(output_dir, f"SOP_LUT_{file_name}.vhd")
        with open(vhdl_file_path, "w") as vhdl_file:
            vhdl_file.write(vhdl_code)
        
        vhdl_code = generate_vhdl_sop_with_dont_care(file_name, grouped_data, binary_length)
        vhdl_file_path = os.path.join(output_dir, f"SOP_LUT_simple_{file_name}.vhd")
        with open(vhdl_file_path, "w") as vhdl_file:
            vhdl_file.write(vhdl_code)
        
        vhdl_code = generate_vhdl_rom(file_name, grouped_data, binary_length)
        vhdl_file_path = os.path.join(output_dir, f"ECC_mem_rom_{file_name}.vhd")
        with open(vhdl_file_path, "w") as vhdl_file:
            vhdl_file.write(vhdl_code)
        
    for i, CHVs in enumerate(groupMemFiles):
        input_file = directory+CHVs
        # print(input_file, i)
        bitvectors = read_mif_file(input_file)
        # print(bitvectors)

        flag_vector = generate_flag_memory_file(bitvectors) 
        write_mif_file("flag_memory", i, flag_vector)
        parity_vector = generate_parity_memory_file(bitvectors, flag_vector)
        write_mif_file("parity_memory", i, parity_vector)
        
        Sizing_the_same("parity_memory", directory)

if __name__ == "__main__":
    main()

[]


In [8]:
merge_mif_files('./hardware/RCEHDC/mnist_example/mem/normal')

Merging completed! Created 2 output files.


In [25]:

# Example usage:
input_file = 'CHV_img.mif'  # Replace with the path to your file
segment_number = 2  # Specify the segment number
generate_flag_memory_file(bitvectors, segment_size, output_file_base)

print(rearranged_bitstreams)

['00000000000000000000000000110000001110011111111111110101111101100111000000000000001110000000011100011000000011000000010000110011100000000000011100010011101111011000000111111000000011001011110000010111111111111111011110000111010000011100011110001111000000111111111001111111010011111001110000001110000000011111111000001111100010111000110000100000000000111001011100000110111111011111110011100100000000001011111011110000000011001110000011111111110100000100011110100000001001011000110010000111111111100000000111110010010001111000110000011011100010010001000011101000011011111101111111001100000011101111110000001110110000011111100000001111111111111111101111101101111111111110010000100110010011110011000011001110011001110111100001111111000111111000000000110000000000010000000100100000011111111111111111001111101100011111111000001110000011110011011101100000111110111111011101111111000100010001110111111001111011111101111011100000111101111111000010000000100111000110000000011111110000000000100011111110001110

In [ ]:
# d//

In [ ]:

# def rearrange_bitstream(file_path, segment_number):
#     """
#     Rearranges bitstreams from a .mif file based on a given segment number.

#     Args:
#         file_path (str): Path to the input .mif file.
#         segment_number (int): Number used to segment the bitstream.

#     Returns:
#         list: A list of rearranged bitstreams.
#     """
#     rearranged_bitstreams = []

#     # Read the .mif file and extract the bitstreams
#     with open(file_path, 'r') as file:
#         lines = file.readlines()

#     # Process each line as a bitstream
#     for line in lines:
#         bitstream = line.strip()  # Remove whitespace or newline
#         if not bitstream:
#             continue
#         bit_length = len(bitstream)

#         # Ensure the bitstream length is compatible with the segment_number
#         if bit_length % segment_number != 0:
#             print(f"Warning: Bitstream length {bit_length} is not divisible by {segment_number}")
#             continue

#         segment_size = bit_length // segment_number
#         rearranged = []

#         # Perform the rearrangement
#         for i in range(segment_size):
#             for j in range(segment_number):
#                 rearranged.append(bitstream[i + j * segment_size])

#         rearranged_bitstreams.append(''.join(rearranged))

#     return rearranged_bitstreams